# How to use capture ratios to improve investment performance

The up-market capture ratio is a way to evaluate how well an investment performs when the market is rising. It compares the performance against a benchmark during bull markets.

The ratio is calculated by dividing the investment’s returns by the returns of the benchmark during a bull market.

The higher the up-market capture ratio, the better the performance during positive market conditions.

If you’re a systematic trader, you can use the ratio to determine if your strategy performs well during bull markets or whether it may be better suited to other market regimes.

In [1]:
# Imports and set up
from openbb_terminal.sdk import openbb

prices = openbb.economy.index(
    ["AAPL", "WMT", "SPY"], 
    start_date="2020-01-01"
)

returns = prices.pct_change().dropna()

data = returns.assign(
    port=returns[["AAPL", "WMT"]].sum(axis=1)
)[["SPY", "port"]]

Forecasting Toolkit is disabled. To use the Forecasting features please install the toolkit following the instructions here: https://docs.openbb.co/sdk/quickstart/installation/

Portfolio Optimization Toolkit is disabled. To use the Optimization features please install the toolkit following the instructions here: https://docs.openbb.co/sdk/quickstart/installation/

# Compute the upside capture ratio

There are two steps to computing the ratio. In the first step, we build a function to compute the annual return.

In [2]:
def annual_return(returns):
    num_years = len(returns) / 252
    return (returns + 1).prod() ** (1 / num_years) - 1

The code divides the number of returns by 252 to get the number of years. The function then computes the cumulative product of the daily returns to get the cumulative return, raises it to the power of the inverse of the number of years, and subtracts 1 to provide the annualized return.

In the second we build a function to compute the upside capture.

In [3]:
def upside_capture(port_returns, bench_returns):
    mask = bench_returns > 0
    port_returns = port_returns[mask]
    bench_returns = bench_returns[mask]

    return (
        annual_return(port_returns) 
        / annual_return(bench_returns)
    )

The ratio is calculated by dividing the investment’s returns by the returns of the benchmark during a rising market. The first step is to find where the benchmark is rising which is done by masking the returns when they are not positive.

Then to compute the up-market capture ratio, divide the annual returns of the positive portfolio returns by the annual return of the positive benchmark returns.

A positive up-market capture ratio indicates this portfolio outperforms the benchmark during bull markets.

In [6]:
data

,SPY,port
Date,,
2020-01-03,-0.007572,-0.018550
2020-01-06,0.003815,0.005932
2020-01-07,-0.002812,-0.013968
2020-01-08,0.005329,0.012655
2020-01-09,0.006781,0.031571
...,...,...
2023-08-07,0.008729,-0.003675
2023-08-08,-0.004349,0.006309
2023-08-09,-0.006685,-0.007336


In [5]:
upside_capture(data["port"], data["SPY"])

6.768824259737526